In [1]:
import yourdfpy
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from urdf_utils import *



asset_path = "/home/grl/repo/RobotsMakingRobots/evolutionary_loop/assets/robogrammar_bank/robot_1085/robot.urdf"

urdf = yourdfpy.URDF.load(asset_path,
                          build_scene_graph=False,
                          load_meshes=False,
                          load_collision_meshes=True,
                          build_collision_scene_graph=True
                          )

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
urdf.base_link

'base_link_0'

In [3]:
urdf.link_map[urdf.base_link]

Link(name='base_link_0', inertial=Inertial(origin=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), mass=0.2667435114503817, inertia=array([[0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1]])), visuals=[Visual(name=None, origin=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), geometry=Geometry(box=None, cylinder=None, sphere=None, mesh=Mesh(filename='meshes/link_0.obj', scale=None)), material=Material(name='gray', color=None, texture=None))], collisions=[Collision(name='base_link_0', origin=array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), geometry=Geometry(box=None, cylinder=None, sphere=None, mesh=Mesh(filename='meshes/link_0.obj', scale=None)))])

In [8]:
link_to_joint_dict = {link_name: [] for link_name in urdf.link_map.keys()}
for joint_name,joint in urdf.joint_map.items():
    print(joint.parent,joint.child)

TypeError: unhashable type: 'list'

In [9]:
[[] for link_name in urdf.link_map.keys()]

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]

In [ ]:
{link_name: [] for link_name in urdf.link_map.keys()}

In [3]:
import os
import re
import glob
# pip install pycollada
# pip instal open3d
import open3d as o3d
import trimesh

def load_to_o3d_mesh(path:str):
    if path.endswith('.dae'): # dae mesh
        import collada
        c_mesh = collada.Collada(path)
        o3d_mesh = o3d.geometry.TriangleMesh()
        for geom in c_mesh.geometries:
            print(geom)
            for triset in geom.primitives:
                if not type(triset) is collada.triangleset.TriangleSet:
                    continue
                print(geom,triset)
            
                o3d_mesh += o3d.geometry.TriangleMesh(
                    o3d.utility.Vector3dVector(triset.vertex), # vertices
                    o3d.utility.Vector3iVector(triset.vertex_index) # triangles
                    )
    else: # stl/obj
        o3d_mesh = o3d.io.read_triangle_mesh(path)
        
    if o3d_mesh.is_watertight()==False: # fix water tightness
        trimesh_mesh = trimesh.load(path) # Load your Trimesh mesh
        # Repair the mesh
        print("mesh is not water tight, tyring to fill holes")
        if trimesh.repair.fill_holes(trimesh_mesh) is False:
            raise(NotImplementedError("cannot fix broken mesh"))
        # Create Open3D TriangleMesh
        o3d_mesh = o3d.geometry.TriangleMesh()
        o3d_mesh.vertices = o3d.utility.Vector3dVector(trimesh_mesh.vertices.astype(np.float64))
        o3d_mesh.triangles = o3d.utility.Vector3iVector(trimesh_mesh.faces)
        
    o3d_mesh.compute_triangle_normals()
    o3d_mesh.compute_vertex_normals()
    return o3d_mesh

def simplify_o3d_mesh(o3d_mesh:o3d.geometry.TriangleMesh, decimation:int,maximum_error=0.0005):
    if decimation<=1:
        return o3d_mesh
    mesh_smp = o3d_mesh\
        .remove_unreferenced_vertices()\
        .remove_duplicated_vertices()\
        .remove_duplicated_triangles()\
        .remove_degenerate_triangles()\
        .remove_non_manifold_edges()\
        .merge_close_vertices(maximum_error)\
        .simplify_vertex_clustering(maximum_error)\
        .simplify_quadric_decimation(
            target_number_of_triangles=int(len(o3d_mesh.triangles)/decimation),
            maximum_error = maximum_error
            )
    mesh_smp.compute_vertex_normals()
    mesh_smp.compute_triangle_normals()
    print(mesh_smp)
    return mesh_smp

In [4]:
decimation = 100 # reduce triangles by this factor

src_dir = "urdf/biped_new_foot/meshes"
dest_dir = "urdf/biped_new_foot/simple_mesh"

# src_dir = "urdf/px2_ot/meshes"
# dest_dir = "urdf/px2_ot/simple_meshes"


src_ext = "stl"
# dest_ext = "stl"

# Compile a case-insensitive regular expression pattern for the extension
pattern = re.compile(re.escape(src_ext), re.IGNORECASE)

# Use glob to recursively find all files in the folder and its subfolders
paths = glob.glob(os.path.join(src_dir, '**'), recursive=True)

# Filter the files based on the case-insensitive extension pattern
paths = [file for file in paths if pattern.search(file)]

# Calculate the relative path between the original path and the matching path
relative_paths = [os.path.relpath(path, src_dir) for path in paths]

# # Print the list of files
# for path in relative_paths:
#     print(path)
    

for path, rel_path in zip(paths,relative_paths):
    
    print(f"loading {rel_path}")
    o3d_mesh = load_to_o3d_mesh(path)
    print(o3d_mesh)
    # print(f"watertight={o3d_mesh.is_watertight()}")
    # o3d.visualization.draw_geometries([o3d_mesh])
    mesh_export = o3d_mesh
    
    mesh_export = simplify_o3d_mesh(o3d_mesh,decimation)
    # export
    # export_path = f"{dest_dir}/{rel_path}.{dest_ext}"
    export_path = f"{dest_dir}/{rel_path}"
    
    
    # Get the folder path of the file path
    folder_path = os.path.dirname(export_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    o3d.io.write_triangle_mesh(export_path,mesh_export,write_vertex_normals=True)
    # o3d.visualization.draw_geometries([mesh_export])
    print(f"saved to \"{export_path}\"")
    file_size_bytes_original = os.path.getsize(path)
    file_size_bytes_export = os.path.getsize(export_path)
    print(f"reduction ratio={(1-file_size_bytes_export/file_size_bytes_original)*100:.1f}%")
    print("-"*10)

loading 2_hip_L_1.stl
mesh is not water tight, tyring to fill holes


NotImplementedError: cannot fix broken mesh